In [15]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from itertools import product
!pip install kmodes

In [16]:
persones=pd.read_csv('Accident_people_2021_2022.csv')
persones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17919 entries, 0 to 17918
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Unnamed: 0                    17919 non-null  int64 
 1   File_Num                      17919 non-null  object
 2   District                      17919 non-null  object
 3   Neighbourhood                 17919 non-null  object
 4   Street_name                   17919 non-null  object
 5   Day_name                      17919 non-null  object
 6   year                          17919 non-null  int64 
 7   month                         17919 non-null  int64 
 8   Month_name                    17919 non-null  object
 9   day                           17919 non-null  int64 
 10  Day_shift                     17919 non-null  object
 11  hour                          17919 non-null  int64 
 12  Pedestrian_situation          17919 non-null  object
 13  Vehicle_type    

In [17]:
weather=pd.read_csv('data_meteo_estudifinal_2021_2022.csv')
weather = weather.drop(weather.columns[0], axis=1)
weather.rename(columns={'DATA_LECTURA':'Date'},inplace=True)
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35040 entries, 0 to 35039
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       35040 non-null  object 
 1   03.TM      35040 non-null  float64
 2   06.HRM     35040 non-null  float64
 3   07.PPT24H  35040 non-null  float64
 4   08.HPA     35040 non-null  float64
 5   09.RS24H   35040 non-null  float64
 6   10.VVM10   35040 non-null  float64
dtypes: float64(6), object(1)
memory usage: 1.9+ MB


In [18]:
weather.head()

,Date,03.TM,06.HRM,07.PPT24H,08.HPA,09.RS24H,10.VVM10
0,2021-01-01 00:00:00,10.1,57.0,0.0,1002.2,0.0,1.6
1,2021-01-01 00:30:00,9.9,59.0,0.0,1002.0,0.0,2.1
2,2021-01-01 01:00:00,9.6,59.0,0.0,1001.9,0.0,2.2
3,2021-01-01 01:30:00,9.6,59.0,0.0,1001.8,0.0,2.3
4,2021-01-01 02:00:00,9.6,59.0,0.0,1001.6,0.0,2.1


In [19]:
#FROM ALL THE DATA WE ARE JUST GOING TO FOCUS N¡ON THE DRIVERS BECAUSE
#THE PASSANGER OR THE PEDESTRIAN DOESN0T HAVE ANY KIND OF POWER TO NOT BE IN THE SITUATION OR HAVE ANY CONTROL

persones=persones[['Day_shift','Vehicle_type','Gender','Age','Date']]
weather=weather[['Date','03.TM','06.HRM','10.VVM10']]

#JOIN THE DATASETS WITH THE VARIABLE DATE
variables = pd.merge(persones, weather, on='Date', how='left')
variables.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 17919 entries, 0 to 17918
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Day_shift     17919 non-null  object 
 1   Vehicle_type  17919 non-null  object 
 2   Gender        17919 non-null  object 
 3   Age           17919 non-null  object 
 4   Date          17919 non-null  object 
 5   03.TM         17919 non-null  float64
 6   06.HRM        17919 non-null  float64
 7   10.VVM10      17919 non-null  float64
dtypes: float64(3), object(5)
memory usage: 1.2+ MB


In [20]:
#add variable quarter and start studying the different variables
variables['Quarter'] = variables['Date'].apply(lambda x: pd.Timestamp(x).quarter)
variables.head()

,Day_shift,Vehicle_type,Gender,Age,Date,03.TM,06.HRM,10.VVM10,Quarter
0,Afternoon,Vehicle without motor,Women,8,2021-04-13 20:00:00,13.8,78.0,0.8,2
1,Morning,Other,Men,58,2021-03-12 12:00:00,13.0,41.0,2.8,1
2,Afternoon,Other,Women,52,2021-10-11 19:00:00,14.9,75.0,2.7,4
3,Afternoon,Vehicle without motor,Men,47,2021-02-06 15:00:00,22.0,77.0,3.3,1
4,Afternoon,Other,Men,4,2021-09-12 17:00:00,10.8,53.0,0.9,3


In [21]:
# LETS TRANSFORM TO CATHEGORIC THE COLUMN EDAT
#/// desconegut(404) -->NOT KNOWN
#transform value desconegut a 404
variables['Age'] = variables['Age'].replace('Desconegut', '404')
def categorize_age(age):
    if int(age) <= 25:
        return '16-25'
    elif int(age) <= 35 and int(age) > 25:
        return '26-35'
    elif int(age) <= 45 and int(age) > 35:
        return '36-45'
    elif int(age) <= 55 and int(age) > 45:
        return '46-55'
    elif age == '404':
        return 'Age not known'
    else:
        return '+56'

# Apply function to transform ages to categories
variables['Age'] = variables['Age'].apply(categorize_age)

x=variables['Age'].unique()
print(x)

# count the frequency of each value in column 'a'
value_counts = variables['Age'].value_counts()

# calculate the percentage of times each value appears
percentage = (value_counts / len(variables)) * 100
print(percentage)

['16-25' '+56' '46-55' '26-35' '36-45' 'Age not known']
26-35            25.771527
16-25            20.620570
36-45            19.917406
+56              17.489815
46-55            16.111390
Age not known     0.089291
Name: Age, dtype: float64


In [22]:
#x=variables['03.TM'].unique()
# print(np.sort(x)) the data goes from 3.8 to 34.2
def cat_temp(temp):
    if int(temp) <= 13:
        return 'Very low'
    elif int(temp) <= 17 and int(temp) > 13:
        return 'Low'
    elif int(temp) <= 24 and int(temp) > 17:
        return 'Mild'
    else:
        return 'Hot'
variables['03.TM'] = variables['03.TM'].apply(cat_temp)

x=variables['03.TM'].unique()
print(x)

# count the frequency of each value in column 'a'
value_counts = variables['03.TM'].value_counts()

# calculate the percentage of times each value appears
percentage = (value_counts / len(variables)) * 100
print(percentage)

['Very low' 'Low' 'Mild' 'Hot']
Mild        30.397902
Very low    24.750265
Hot         23.405324
Low         21.446509
Name: 03.TM, dtype: float64


In [23]:
#x=variables['06.HRM'].unique()
#print(np.sort(x)) from 9 percent to 100 percent
def cat_humidity(hum):
    if int(hum) <= 48:
        return 'Low humidity'
    elif int(hum) <= 60 and int(hum) > 48:
        return 'Moderate humidity'
    elif int(hum) <= 75 and int(hum) > 60:
        return 'High humidity'
    else:
        return 'Very high'
variables['06.HRM'] = variables['06.HRM'].apply(cat_humidity)

x=variables['06.HRM'].unique()
print(x)

# count the frequency of each value in column 'a'
value_counts = variables['06.HRM'].value_counts()

# calculate the percentage of times each value appears
percentage = (value_counts / len(variables)) * 100
print(percentage)

['Very high' 'Low humidity' 'High humidity' 'Moderate humidity']
High humidity        39.092583
Moderate humidity    25.514817
Low humidity         18.907305
Very high            16.485295
Name: 06.HRM, dtype: float64


In [24]:
#x=variables['10.VVM10'].unique()
#print(np.sort(x)) from 0 to 8.7
def cat_wind(wind):
    if int(wind) <= 1:
        return 'Low speed'
    elif int(wind) <= 2 and int(wind) > 1:
        return 'Moderate speed'
    else:
        return 'High speed'
    
variables['10.VVM10'] = variables['10.VVM10'].apply(cat_wind)

x=variables['10.VVM10'].unique()
print(x)

# count the frequency of each value in column 'a'
value_counts = variables['10.VVM10'].value_counts()

# calculate the percentage of times each value appears
percentage = (value_counts / len(variables)) * 100
print(percentage)

['Low speed' 'Moderate speed' 'High speed']
Low speed         43.211117
Moderate speed    30.157933
High speed        26.630950
Name: 10.VVM10, dtype: float64


In [25]:
column_names = variables.columns.tolist()
print(column_names)
categorical=column_names

['Day_shift', 'Vehicle_type', 'Gender', 'Age', 'Date', '03.TM', '06.HRM', '10.VVM10', 'Quarter']


In [26]:
tuple(categorical)

('Day_shift',
 'Vehicle_type',
 'Gender',
 'Age',
 'Date',
 '03.TM',
 '06.HRM',
 '10.VVM10',
 'Quarter')

In [27]:
import pandas as pd

# Load the dataset into a Pandas DataFrame
df = variables

# Count the number of occurrences of each unique combination of values
counts = df.groupby(categorical).size().reset_index(name='count')

# Get the top 10 combinations of values that are the most repeated in the dataset
top_counts = counts.sort_values('count', ascending=False).head(10)

# Print the top 10 combinations of values
print(top_counts)

       Day_shift Vehicle_type Gender    Age                 Date     03.TM  \
13981    Morning        Other  Women    +56  2022-12-22 11:00:00       Low   
13110    Morning        Other    Men    +56  2022-05-10 11:00:00      Mild   
263    Afternoon          Car    Men  16-25  2022-04-02 17:00:00  Very low   
13028    Morning        Other    Men    +56  2021-06-05 11:00:00      Mild   
9059     Morning          Car    Men  36-45  2022-06-10 08:00:00      Mild   
3128   Afternoon   Motorcycle    Men  26-35  2022-06-08 18:00:00       Hot   
13802    Morning        Other  Women    +56  2021-10-27 13:00:00      Mild   
13760    Morning        Other  Women    +56  2021-07-14 13:00:00      Mild   
15136      Night          Car    Men  46-55  2022-10-16 02:00:00      Mild   
10964    Morning   Motorcycle    Men  26-35  2022-12-21 08:00:00  Very low   

                  06.HRM        10.VVM10  Quarter  count  
13981  Moderate humidity       Low speed        4      6  
13110      High humidit

In [28]:
import pandas as pd
import itertools 

#Create a list of diferent combinations of 3 variables of the given variables in categorical
combinations = list(itertools.combinations(categorical, 3))

# Load the dataset into a Pandas DataFrame
df = variables

#Create a dictonary that has the results for each combination of variables

dic={}

# Count the number of occurrences of each unique combination of values
for combination in combinations:
    counts = df.groupby(list(combination)).size().reset_index(name='count')
    counts['percentage %'] = counts['count'] / df.shape[0] * 100

    # Get the top 10 combinations of values that are the most repeated in the dataset
    top_counts = counts.sort_values('count', ascending=False).head(10)
    dic[tuple(combination)]=counts.sort_values('count', ascending=False)
dic3=dic
# It gives the combinations of values that are freqüent in more that 10 percent of all the accidents.
for key, value in dic.items():
    condition = value.iloc[:, -1] > 10
    result = value[condition]
    
    if not result.empty:
        print(result)
        print('-----------------------------------------------------')

    Day_shift Vehicle_type Gender  count  percentage %
3   Afternoon   Motorcycle    Men   3300     18.416206
12    Morning   Motorcycle    Men   2276     12.701602
-----------------------------------------------------
   Day_shift Vehicle_type         06.HRM  count  percentage %
4  Afternoon   Motorcycle  High humidity   1976     11.027401
-----------------------------------------------------
   Day_shift Vehicle_type   10.VVM10  count  percentage %
4  Afternoon   Motorcycle  Low speed   1933     10.787432
-----------------------------------------------------
   Day_shift Gender         06.HRM  count  percentage %
0  Afternoon    Men  High humidity   2175     12.137954
-----------------------------------------------------
   Day_shift Gender   10.VVM10  count  percentage %
1  Afternoon    Men  Low speed   2104     11.741727
-----------------------------------------------------
   Vehicle_type Gender    Age  count  percentage %
15   Motorcycle    Men  26-35   1803     10.061945
-------

In [29]:
import pandas as pd
import itertools 

#Create a list of diferent combinations of 4 variables of the given variables in categorical
combinations = list(itertools.combinations(categorical, 4))

# Load the dataset into a Pandas DataFrame
df = variables

#Create a dictonary of the top 10 counts for each combination

dic={}

# Count the number of occurrences of each unique combination of values
for combination in combinations:
    counts = df.groupby(list(combination)).size().reset_index(name='count')
    counts['percentage %'] = counts['count'] / df.shape[0] * 100

    # Get the top 10 combinations of values that are the most repeated in the dataset
    top_counts = counts.sort_values('count', ascending=False).head(10)
    dic[tuple(combination)]=counts.sort_values('count', ascending=False)
dic4=dic 
# It gives the combinations of 4 values that are freqüent in more that 5 percent of all the accidents.
for key, value in dic.items():
    condition = value.iloc[:, -1] > 5
    result = value[condition]
    
    if not result.empty:
        print(result)
        print('-----------------------------------------------------')

    Day_shift Vehicle_type Gender    Age  count  percentage %
14  Afternoon   Motorcycle    Men  26-35    969      5.407668
-----------------------------------------------------
    Day_shift Vehicle_type Gender 03.TM  count  percentage %
11  Afternoon   Motorcycle    Men  Mild    992      5.536023
-----------------------------------------------------
   Day_shift Vehicle_type Gender         06.HRM  count  percentage %
9  Afternoon   Motorcycle    Men  High humidity   1296      7.232546
-----------------------------------------------------
   Day_shift Vehicle_type Gender        10.VVM10  count  percentage %
8  Afternoon   Motorcycle    Men       Low speed   1243      6.936771
9  Afternoon   Motorcycle    Men  Moderate speed   1094      6.105251
7  Afternoon   Motorcycle    Men      High speed    963      5.374184
-----------------------------------------------------
    Day_shift Vehicle_type Gender  Quarter  count  percentage %
10  Afternoon   Motorcycle    Men        2    909      5

In [30]:
import pandas as pd
import itertools 

#Create a list of diferent combinations of 5 variables of the given variables in categorical
combinations = list(itertools.combinations(categorical, 5))

# Load the dataset into a Pandas DataFrame
df = variables

#Create a dictonary of the top 10 counts for each combination

dic={}

# Count the number of occurrences of each unique combination of values
for combination in combinations:
    counts = df.groupby(list(combination)).size().reset_index(name='count')
    counts['percentage %'] = counts['count'] / df.shape[0] * 100

    # Get the top 10 combinations of values that are the most repeated in the dataset
    top_counts = counts.sort_values('count', ascending=False).head(10)
    dic[tuple(combination)]=counts.sort_values('count', ascending=False)
dic5=dic    
# It gives the combinations of 5 values that are freqüent in more that 2 percent of all the accidents.
for key, value in dic.items():
    condition = value.iloc[:, -1] > 2
    result = value[condition]
    
    if not result.empty:
        print(result)
        print('-----------------------------------------------------')

    Day_shift Vehicle_type Gender    Age         06.HRM  count  percentage %
51  Afternoon   Motorcycle    Men  26-35  High humidity    379      2.115073
-----------------------------------------------------
    Day_shift Vehicle_type Gender    Age   10.VVM10  count  percentage %
40  Afternoon   Motorcycle    Men  26-35  Low speed    383      2.137396
-----------------------------------------------------
    Day_shift Vehicle_type Gender 03.TM         06.HRM  count  percentage %
41  Afternoon   Motorcycle    Men  Mild  High humidity    401      2.237848
-----------------------------------------------------
    Day_shift Vehicle_type Gender     03.TM   10.VVM10  count  percentage %
35  Afternoon   Motorcycle    Men  Very low  Low speed    440      2.455494
32  Afternoon   Motorcycle    Men      Mild  Low speed    363      2.025783
-----------------------------------------------------
    Day_shift Vehicle_type Gender     03.TM  Quarter  count  percentage %
35  Afternoon   Motorcycle    

In [31]:
import pandas as pd
import itertools 

#Create a list of diferent combinations of 6 variables of the given variables in categorical
combinations = list(itertools.combinations(categorical, 6))

# Load the dataset into a Pandas DataFrame
df = variables

#Create a dictonary of the top 10 counts for each combination

dic={}

# Count the number of occurrences of each unique combination of values
for combination in combinations:
    counts = df.groupby(list(combination)).size().reset_index(name='count')
    counts['percentage %'] = counts['count'] / df.shape[0] * 100

    # Get the top 10 combinations of values that are the most repeated in the dataset
    top_counts = counts.sort_values('count', ascending=False).head(10)
    dic[tuple(combination)]=counts.sort_values('count', ascending=False)
dic6=dic   
# It gives the combinations of 6 values that are freqüent in more that 1 percent of all the accidents.
for key, value in dic.items():
    condition = value.iloc[:, -1] > 1
    result = value[condition]
    
    if not result.empty:
        print(result)
        print('-----------------------------------------------------')

     Day_shift Vehicle_type Gender 03.TM         06.HRM  Quarter  count  \
128  Afternoon   Motorcycle    Men   Hot  High humidity        3    191   

     percentage %  
128      1.065908  
-----------------------------------------------------
     Day_shift Vehicle_type Gender         06.HRM   10.VVM10  Quarter  count  \
104  Afternoon   Motorcycle    Men  High humidity  Low speed        4    207   

     percentage %  
104      1.155198  
-----------------------------------------------------


In [32]:
combinations = list(itertools.combinations(categorical, 3))
print(combinations)

[('Day_shift', 'Vehicle_type', 'Gender'), ('Day_shift', 'Vehicle_type', 'Age'), ('Day_shift', 'Vehicle_type', 'Date'), ('Day_shift', 'Vehicle_type', '03.TM'), ('Day_shift', 'Vehicle_type', '06.HRM'), ('Day_shift', 'Vehicle_type', '10.VVM10'), ('Day_shift', 'Vehicle_type', 'Quarter'), ('Day_shift', 'Gender', 'Age'), ('Day_shift', 'Gender', 'Date'), ('Day_shift', 'Gender', '03.TM'), ('Day_shift', 'Gender', '06.HRM'), ('Day_shift', 'Gender', '10.VVM10'), ('Day_shift', 'Gender', 'Quarter'), ('Day_shift', 'Age', 'Date'), ('Day_shift', 'Age', '03.TM'), ('Day_shift', 'Age', '06.HRM'), ('Day_shift', 'Age', '10.VVM10'), ('Day_shift', 'Age', 'Quarter'), ('Day_shift', 'Date', '03.TM'), ('Day_shift', 'Date', '06.HRM'), ('Day_shift', 'Date', '10.VVM10'), ('Day_shift', 'Date', 'Quarter'), ('Day_shift', '03.TM', '06.HRM'), ('Day_shift', '03.TM', '10.VVM10'), ('Day_shift', '03.TM', 'Quarter'), ('Day_shift', '06.HRM', '10.VVM10'), ('Day_shift', '06.HRM', 'Quarter'), ('Day_shift', '10.VVM10', 'Quarter')